# Momentum vs Mean-Reversion — Research Notebook
This notebook compares a momentum moving-average crossover against an RSI-based mean-reversion strategy on SPY.
It will try to use `yfinance` to download SPY; if unavailable, it falls back to the provided `data/SPY_sample.csv`.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, os
from pathlib import Path

DATA = Path('..')/'data'
RESULTS = Path('..')/'results'
RESULTS.mkdir(parents=True, exist_ok=True)

def load_data():
    try:
        import yfinance as yf
        df = yf.download('SPY', start='2018-01-01', progress=False)
        df = df[['Open','High','Low','Close','Volume']].reset_index().rename(columns={'Date':'Date'})
        print('Loaded SPY via yfinance.')
        return df
    except Exception as e:
        print('yfinance unavailable; using local CSV.')
        return pd.read_csv(DATA/'SPY_sample.csv', parse_dates=['Date'])

df = load_data()
df = df.sort_values('Date')
df.head()

In [ ]:
# Helpers
def ma_signal(close: pd.Series, fast:int, slow:int) -> pd.Series:
    sf = close.rolling(fast).mean()
    ss = close.rolling(slow).mean()
    sig = (sf > ss).astype(int)
    return sig.shift(1).fillna(0)

def rsi(series: pd.Series, period:int=14) -> pd.Series:
    delta = series.diff()
    up = (delta.clip(lower=0)).ewm(alpha=1/period, adjust=False).mean()
    down = (-delta.clip(upper=0)).ewm(alpha=1/period, adjust=False).mean()
    rs = up / (down.replace(0, np.nan))
    rsi = 100 - (100/(1+rs))
    return rsi

def mean_reversion_signal(close: pd.Series, lb:int=14, low_th:float=30, high_th:float=70) -> pd.Series:
    r = rsi(close, lb)
    sig = (r < low_th).astype(int)
    return sig.shift(1).fillna(0)

def vol_target_weights(returns: pd.Series, target_annual_vol=0.15, lookback=30) -> pd.Series:
    vol = returns.rolling(lookback).std()
    ann_vol = vol*np.sqrt(252)
    w = (target_annual_vol/ann_vol).clip(upper=1.0)
    return w.fillna(0)

def backtest(close: pd.Series, signal: pd.Series, costs_bps=5, use_vol_target=True):
    ret = close.pct_change().fillna(0)
    if use_vol_target:
        w = vol_target_weights(ret)
        pos = signal * w
    else:
        pos = signal
    gross = pos * ret
    turnover = pos.diff().abs().fillna(0)
    costs = turnover * (costs_bps/1e4)
    net = gross - costs
    equity = (1+net).cumprod()
    return net, equity, float(turnover.sum())

def summarize(returns: pd.Series):
    n = len(returns)
    if n == 0:
        return dict(CAGR=0, Vol=0, Sharpe=0, MaxDD=0)
    ann = (1+returns).prod()**(252/n) - 1
    vol = returns.std()*np.sqrt(252)
    sharpe = (ann/vol) if vol>0 else 0
    curve = (1+returns).cumprod()
    peak = curve.cummax()
    mdd = float((curve/peak - 1).min())
    return dict(CAGR=float(ann), Vol=float(vol), Sharpe=float(sharpe), MaxDD=mdd)

In [ ]:
# Prepare series and split
px = df.set_index('Date')['Close']
split = int(len(px)*0.7)
train, test = px.iloc[:split], px.iloc[split:]

# Momentum grid
grid = [(10,50),(20,100),(50,200)]
rows = []
for fast, slow in grid:
    sig_tr = ma_signal(train, fast, slow)
    ret_tr, eq_tr, _ = backtest(train, sig_tr, costs_bps=5, use_vol_target=True)
    rows.append({'fast':fast, 'slow':slow, **summarize(ret_tr)})
res_train = pd.DataFrame(rows).sort_values('Sharpe', ascending=False)
best_fast, best_slow = int(res_train.iloc[0]['fast']), int(res_train.iloc[0]['slow'])
res_train

In [ ]:
# Evaluate on test
sig_test_mom = ma_signal(test, best_fast, best_slow)
ret_mom, eq_mom, turn_mom = backtest(test, sig_test_mom, costs_bps=5, use_vol_target=True)
met_mom = summarize(ret_mom)

sig_test_mr = mean_reversion_signal(test, lb=14, low_th=30, high_th=70)
ret_mr, eq_mr, turn_mr = backtest(test, sig_test_mr, costs_bps=5, use_vol_target=True)
met_mr = summarize(ret_mr)

sig_bh = pd.Series(1, index=test.index)
ret_bh, eq_bh, _ = backtest(test, sig_bh, costs_bps=0, use_vol_target=False)
met_bh = summarize(ret_bh)

met_mom, met_mr, met_bh

In [ ]:
# Equity curves
plt.figure()
eq_bh.plot(label='Buy & Hold')
eq_mom.plot(label=f'Momentum ({best_fast},{best_slow})')
eq_mr.plot(label='Mean-Reversion (RSI)')
plt.title('Equity Curves (Test Period)')
plt.xlabel('Date'); plt.ylabel('Equity')
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS/'equity_curves.png')
RESULTS/'equity_curves.png'

In [ ]:
# Drawdown for momentum
curve = eq_mom
peak = curve.cummax()
dd = curve/peak - 1
plt.figure()
dd.plot()
plt.title('Drawdown — Momentum (Test)')
plt.xlabel('Date'); plt.ylabel('Drawdown')
plt.tight_layout()
plt.savefig(RESULTS/'drawdown_momentum.png')
RESULTS/'drawdown_momentum.png'